<a href="https://colab.research.google.com/github/sumaiya08/medicaldata/blob/master/EDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pandas.util.testing as tm
sns.set()
# Access data using Google BigQuery.
from google.colab import auth
from google.cloud import bigquery

In [0]:
auth.authenticate_user()
print('Authenticated')


Authenticated


In [0]:
project_id='eicudata'

Total number of hospitals in the database -

In [0]:
%%bigquery --project $project_id totalhosp
Select COUNT (DISTINCT hospitalid)
from `physionet-data.eicu_crd.hospital`;

In [0]:
totalhosp

,f0_
0,208


Total number of ICU'S - 

In [0]:
%%bigquery --project $project_id totalicu
SELECT COUNT (DISTINCT wardid )
FROM `physionet-data.eicu_crd.patient`


In [0]:
totalicu

,f0_
0,335


Procedures that have ICD 9/10 codes available -

In [0]:
%%bigquery --project $project_id icdcode
select diagnosisstring,icd9code,count(*) as count
from `physionet-data.eicu_crd.diagnosis` 
where icd9code is not null
group by diagnosisstring, icd9code


In [0]:
icdcode

,diagnosisstring,icd9code,count
0,cardiovascular|shock / hypotension|hypovolemic...,"785.59, R57.1, R58",3249
1,cardiovascular|diseases of the aorta|aortic an...,"441.4, I71.4",727
2,cardiovascular|diseases of the aorta|aortic an...,"441.9, I71.9",81
3,pulmonary|disorders of the airways|asthma / br...,"493.90, J45",7593
4,oncology|hematologic malignancy|leukemia|chron...,"204.10, C91.10",394
...,...,...,...
3928,renal|disorder of acid base|metabolic acidosis...,"276.2, E87.2",5
3929,surgery|neurosurgical issues|seizures|from tumor,"345.90, R56.9",1
3930,pulmonary|respiratory failure|acute respirator...,"518.81, J96.00",1
3931,surgery|respiratory failure|ventilatory failur...,"518.81, J96.00",1


Top 10 diseases common amongst patients -

In [0]:
%%bigquery --project $project_id topdiseases 
select apacheadmissiondx, count(apacheadmissiondx) as count
from `physionet-data.eicu_crd.patient` 
group by apacheadmissiondx 
order by count(apacheadmissiondx) desc
limit 10;

In [0]:
topdiseases

,apacheadmissiondx,count
0,,22996
1,"Sepsis, pulmonary",8862
2,"Infarction, acute myocardial (MI)",7228
3,"CVA, cerebrovascular accident/stroke",6647
4,"CHF, congestive heart failure",6617
5,"Sepsis, renal/UTI (including bladder)",5273
6,"Rhythm disturbance (atrial, supraventricular)",4827
7,Diabetic ketoacidosis,4825
8,Cardiac arrest (with or without respiratory ar...,4580
9,"CABG alone, coronary artery bypass grafting",4543


Identifying patients with heart diseases using related drug name - 

In [0]:
%%bigquery --project $project_id heartpatients 
select gender, age, ethnicity,apacheadmissiondx,drugname
from `physionet-data.eicu_crd.admissiondrug` a
join `physionet-data.eicu_crd.patient` b ON (
a.patientunitstayid =b.patientunitstayid
)
where drugname like '%CLOPIDOGREL%' # can also look for drug WARFARIN

In [0]:
heartpatients

,gender,age,ethnicity,apacheadmissiondx,drugname
0,Male,74,,"Endarterectomy, carotid",CLOPIDOGREL ...
1,Male,68,Hispanic,"Bleeding, upper GI",CLOPIDOGREL ...
2,Female,74,,"Endarterectomy, carotid",CLOPIDOGREL ...
3,Male,69,Caucasian,"Hematoma subdural, surgery for",CLOPIDOGREL ...
4,Male,53,Caucasian,Hemorrhage (for gastrointestinal bleeding GI-s...,CLOPIDOGREL ...
...,...,...,...,...,...
1724,Female,85,African American,"Hematologic medical, other",CLOPIDOGREL ...
1725,Male,77,Caucasian,"Infarction, acute myocardial (MI)",CLOPIDOGREL ...
1726,Male,73,African American,Seizures (primary-no structural brain disease),CLOPIDOGREL ...
1727,Male,73,African American,"CVA, cerebrovascular accident/stroke",CLOPIDOGREL ...


Indentifying diabetic patients using drug name -

In [0]:
%%bigquery --project $project_id diabpatients 
select gender,age,ethnicity,apacheadmissiondx,admissionheight,drugname
from `physionet-data.eicu_crd.admissiondrug` a
join `physionet-data.eicu_crd.patient` b ON (
a.patientunitstayid =b.patientunitstayid
)
where drugname like '%GLUCAGON%'    #can also look for drug HUMULIN

In [0]:
diabpatients

,gender,age,ethnicity,apacheadmissiondx,admissionheight,drugname
0,Female,71,African American,"Embolus, pulmonary",162.6,"GLUCAGON,HUMAN RECOMBINANT ..."
1,Female,71,African American,"Embolus, pulmonary",162.6,"GLUCAGON,HUMAN RECOMBINANT ..."
2,Female,71,African American,"Embolus, pulmonary",162.6,"GLUCAGON,HUMAN RECOMBINANT ..."
3,Female,71,African American,"Embolus, pulmonary",162.6,"GLUCAGON,HUMAN RECOMBINANT ..."
4,Female,71,African American,"Embolus, pulmonary",162.6,"GLUCAGON,HUMAN RECOMBINANT ..."
...,...,...,...,...,...,...
143,Female,78,Asian,Emphysema/bronchitis,154.9,GLUCAGON EMERGENCY KIT ...
144,Female,53,Caucasian,Anaphylaxis,177.8,GLUCAGON EMERGENCY KIT ...
145,Male,60,Caucasian,Seizures (primary-no structural brain disease),170.2,GLUCAGON EMERGENCY KIT ...
146,Male,22,Other/Unknown,"Overdose, street drugs (opiates, cocaine, amph...",170.2,GLUCAGON EMERGENCY KIT ...


Types of procedures patients come for -

In [0]:
%%bigquery --project $project_id procedures 
select treatmentstring, count (treatmentstring) as count
from `physionet-data.eicu_crd.treatment` 
group by treatmentstring
order by count(treatmentstring) desc
limit 5;

In [0]:
procedures

,treatmentstring,count
0,pulmonary|ventilation and oxygenation|mechanic...,117481
1,pulmonary|radiologic procedures / bronchoscopy...,65148
2,neurologic|pain / agitation / altered mentatio...,46055
3,renal|urinary catheters|foley catheter,40672
4,cardiovascular|intravenous fluid|normal saline...,40522


Identifying patients that underwent heart procedures -

In [0]:
%%bigquery --project $project_id heartprocedure 
select gender,age,ethnicity,apacheadmissiondx,treatmentstring
from `physionet-data.eicu_crd.treatment`  a
join `physionet-data.eicu_crd.patient` b ON (
a.patientunitstayid =b.patientunitstayid
)
where a.treatmentstring  like '%cardio%'

In [0]:
heartprocedure

,gender,age,ethnicity,apacheadmissiondx,treatmentstring
0,Female,75,Other/Unknown,Emphysema/bronchitis,cardiovascular|arrhythmias|digoxin
1,Female,75,Other/Unknown,Emphysema/bronchitis,cardiovascular|arrhythmias|digoxin
2,Female,75,Other/Unknown,Emphysema/bronchitis,cardiovascular|arrhythmias|calcium channel blo...
3,Female,75,Other/Unknown,Emphysema/bronchitis,cardiovascular|arrhythmias|calcium channel blo...
4,Male,46,Caucasian,"Pneumonia, other",cardiovascular|non-operative procedures|diagno...
...,...,...,...,...,...
860530,Male,74,Caucasian,Hematomas,cardiovascular|hypertension|vasodilating agent...
860531,Male,74,Caucasian,Hematomas,cardiovascular|myocardial ischemia / infarctio...
860532,Male,74,Caucasian,Hematomas,cardiovascular|intravenous fluid|Lactated Ring...
860533,Male,76,Caucasian,Encephalopathies (excluding hepatic),cardiovascular|intravenous fluid|normal saline...


Indentifying heart patients that die before being released from their stay -

In [0]:
%%bigquery --project $project_id death_heart
select gender, age, ethnicity,apacheadmissiondx,hospitaldischargestatus
from `physionet-data.eicu_crd.patient` 
where apacheadmissiondx like '%Cardi%' and hospitaldischargestatus like '%Expire%';

In [0]:
death_heart

,gender,age,ethnicity,apacheadmissiondx,hospitaldischargestatus
0,Male,88,Caucasian,"Cardiovascular medical, other",Expired
1,Female,63,African American,Cardiac arrest (with or without respiratory ar...,Expired
2,Male,84,Caucasian,Cardiac arrest (with or without respiratory ar...,Expired
3,Male,80,Hispanic,Cardiac arrest (with or without respiratory ar...,Expired
4,Male,59,Caucasian,Cardiac arrest (with or without respiratory ar...,Expired
...,...,...,...,...,...
2461,Male,83,Caucasian,Cardiac arrest (with or without respiratory ar...,Expired
2462,Female,78,Caucasian,Cardiac arrest (with or without respiratory ar...,Expired
2463,Male,86,Caucasian,Cardiac arrest (with or without respiratory ar...,Expired
2464,Male,88,Caucasian,Cardiac arrest (with or without respiratory ar...,Expired


In [0]:
print('Number of Heart patients that die during their stay in the hospital - ', len(death_heart))

Number of Heart patients that die during their stay in the hospital -  2466


Identifying diabetic patients that die before being released from the stay -

In [0]:
%%bigquery --project $project_id death_diab
select gender, age, ethnicity,apacheadmissiondx,hospitaldischargestatus
from `physionet-data.eicu_crd.patient` 
where apacheadmissiondx like '%Diab%' and hospitaldischargestatus like '%Expire%'

In [0]:
death_diab

,gender,age,ethnicity,apacheadmissiondx,hospitaldischargestatus
0,Male,28,Caucasian,Diabetic ketoacidosis,Expired
1,Male,36,Hispanic,Diabetic ketoacidosis,Expired
2,Female,44,Hispanic,Diabetic ketoacidosis,Expired
3,Female,53,Hispanic,Diabetic hyperglycemic hyperosmolar nonketotic...,Expired
4,Female,35,Caucasian,Diabetic ketoacidosis,Expired
5,Female,64,Caucasian,Diabetic ketoacidosis,Expired
6,Male,28,Caucasian,Diabetic ketoacidosis,Expired
7,Male,44,Caucasian,Diabetic ketoacidosis,Expired
8,Female,89,Caucasian,Diabetic hyperglycemic hyperosmolar nonketotic...,Expired
9,Female,63,Caucasian,Diabetic ketoacidosis,Expired


In [0]:
print("Number of Diabetic patients that die before being released from their stay -", len(death_diab))

Number of Diabetic patients that die before being released from their stay - 39


Looking for data hiccup -

In [0]:
%%bigquery --project $project_id data_hic
select uniquepid, count(patienthealthsystemstayid) as count_hosp,SUM(unitvisitnumber) as totalicuvisit
from `physionet-data.eicu_crd.patient` 
group by uniquepid
order by uniquepid

In [0]:
data_hic

,uniquepid,count_hosp,totalicuvisit
0,002-10009,1,1
1,002-10018,1,1
2,002-10034,1,1
3,002-10050,3,4
4,002-10052,1,1
...,...,...,...
139362,035-9957,1,1
139363,035-9959,1,1
139364,035-996,1,1
139365,035-9966,1,1


In [0]:
rslt_df = data_hic[data_hic['count_hosp'] > data_hic['totalicuvisit']]
print('\n Data hiccup \n', rslt_df)


 Data hiccup 
 Empty DataFrame
Columns: [uniquepid, count_hosp, totalicuvisit]
Index: []


### Average length of stay for a patient in hospital - 



In [0]:
%%bigquery --project $project_id lengthofstay 
select patientunitstayid,patienthealthsystemstayid,(hospitaldischargeoffset-hospitaladmitoffset)/60/24 AS totalstayindays, apacheadmissiondx,uniquepid
from `physionet-data.eicu_crd.patient` 
order by uniquepid

In [0]:
lengthofstay.info()
lengthofstay

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200859 entries, 0 to 200858
Data columns (total 5 columns):
 #   Column                     Non-Null Count   Dtype  
---  ------                     --------------   -----  
 0   patientunitstayid          200859 non-null  int64  
 1   patienthealthsystemstayid  200859 non-null  int64  
 2   totalstayindays            200859 non-null  float64
 3   apacheadmissiondx          200859 non-null  object 
 4   uniquepid                  200859 non-null  object 
dtypes: float64(1), int64(2), object(2)
memory usage: 7.7+ MB


,patientunitstayid,patienthealthsystemstayid,totalstayindays,apacheadmissiondx,uniquepid
0,224606,193705,5.806944,"GI perforation/rupture, surgery for",002-10009
1,204602,178200,16.508333,"Cardiovascular medical, other",002-10018
2,157016,141169,5.210417,"GI medical, other",002-10034
3,221004,190893,6.883333,,002-10050
4,211144,183274,6.780556,Aortic valve replacement (isolated),002-10050
...,...,...,...,...,...
200854,3351785,2741786,3.710417,Head only trauma,035-9957
200855,3340321,2731423,2.689583,"Infarction, acute myocardial (MI)",035-9959
200856,3345874,2736458,7.965278,Rhythm disturbance (conduction defect),035-996
200857,3352628,2742533,2.226389,"Apnea-sleep; surgery for (i.e., UPPP - uvulopa...",035-9966


### Dropping duplicate values for the same hospitalisation

In [0]:
lengthofstay.drop_duplicates(subset='patienthealthsystemstayid',keep='first',inplace=True)
lengthofstay

,patientunitstayid,patienthealthsystemstayid,totalstayindays,apacheadmissiondx,uniquepid
0,224606,193705,5.806944,"GI perforation/rupture, surgery for",002-10009
1,204602,178200,16.508333,"Cardiovascular medical, other",002-10018
2,157016,141169,5.210417,"GI medical, other",002-10034
3,221004,190893,6.883333,,002-10050
4,211144,183274,6.780556,Aortic valve replacement (isolated),002-10050
...,...,...,...,...,...
200854,3351785,2741786,3.710417,Head only trauma,035-9957
200855,3340321,2731423,2.689583,"Infarction, acute myocardial (MI)",035-9959
200856,3345874,2736458,7.965278,Rhythm disturbance (conduction defect),035-996
200857,3352628,2742533,2.226389,"Apnea-sleep; surgery for (i.e., UPPP - uvulopa...",035-9966


### Hospital stays over 400 days

In [0]:
lengthofstay[lengthofstay['totalstayindays']>400]

,patientunitstayid,patienthealthsystemstayid,totalstayindays,apacheadmissiondx,uniquepid
18288,324726,280163,731.280556,"Vascular surgery, other",004-15917
19047,366449,314457,1096.877778,"Overdose, other toxin, poison or drug",004-25072
25839,530777,448138,600.768056,"CVA, cerebrovascular accident/stroke",005-33644
70522,1072288,797279,633.057639,"Vascular medical, other",009-62
115909,1823388,1435255,413.519444,Pelvis/extremity trauma,018-29216
156984,2747640,2211954,506.520139,Head only trauma,027-195462
200132,3346975,2737468,763.275000,Cardiac arrest (with or without respiratory ar...,035-6284
200133,3351609,2741629,763.455556,"Sepsis, GI",035-6284


Average length of Hospital stay (hospitalisation) and finding median value -(in days)

In [0]:
lengthofstay['totalstayindays'].mean()

7.464949113174917

In [0]:
lengthofstay['totalstayindays'].median()

5.054166666666666

### Finding average length of stay in ICU

In [0]:
%%bigquery --project $project_id los_i
select patientunitstayid,(unitdischargeoffset-hospitaladmitoffset)/60/24 AS totalstayindays, apacheadmissiondx
from `physionet-data.eicu_crd.patient` 

In [0]:
los_i

,patientunitstayid,totalstayindays,apacheadmissiondx
0,3036927,0.166667,"Cardiovascular medical, other"
1,3076905,0.000000,"ARDS-adult respiratory distress syndrome, non-..."
2,486998,0.358333,"Pneumonia, other"
3,3043878,0.109722,Abdomen only trauma
4,3125095,1.487500,Liver transplant
...,...,...,...
200854,1150928,0.367361,Coma/change in level of consciousness (for hep...
200855,1183913,3.119444,Coma/change in level of consciousness (for hep...
200856,1203462,0.840972,Coma/change in level of consciousness (for hep...
200857,3024410,26.475694,Coma/change in level of consciousness (for hep...


In [0]:
los_i.totalstayindays.median()

2.279166666666667

In [0]:
los_i.totalstayindays.mean()

4.317779263065072

Average length of stay for a heart patient in ICU -

In [0]:
%%bigquery --project $project_id stay_heart
select patienthealthsystemstayid, (unitdischargeoffset-hospitaladmitoffset)/60/24 AS lengthofstay, uniquepid
from `physionet-data.eicu_crd.patient`
where apacheadmissiondx like '%Cardi%' 
ORDER BY uniquepid

In [0]:
stay_heart

,patienthealthsystemstayid,lengthofstay,uniquepid
0,178200,4.201389,002-10018
1,202765,10.308333,002-10308
2,139618,8.779167,002-10665
3,204487,0.822917,002-11232
4,139664,9.372222,002-11370
...,...,...,...
6412,2728674,1.540278,035-9585
6413,2738220,2.200000,035-9599
6414,2727436,7.116667,035-9799
6415,2741806,0.576389,035-9825


In [0]:
print('Average length of stay for a heart patient ',stay_heart['lengthofstay'].mean(),'days')
print(f'Maximum length of stay for a heart patient: {stay_heart.lengthofstay.max()} days')

Average length of stay for a heart patient  5.841445682475378 days
Maximum length of stay for a heart patient: 165.94166666666666 days


## Average length of ICU stay for a diabetic patient - 

In [0]:
%%bigquery --project $project_id stay_diab 
select patienthealthsystemstayid, (unitdischargeoffset-hospitaladmitoffset)/60/24 AS lengthofstay, uniquepid
from `physionet-data.eicu_crd.patient` 
where apacheadmissiondx like '%diab%'
ORDER BY uniquepid

In [0]:
stay_diab

,patienthealthsystemstayid,lengthofstay,uniquepid
0,134042,1.653472,002-10424
1,150300,1.025694,002-10541
2,143252,4.046528,002-10669
3,163687,4.171528,002-12088
4,180195,2.222917,002-12387
...,...,...,...
2897,2731472,4.254167,035-9445
2898,2733190,16.348611,035-9582
2899,2731083,2.338194,035-969
2900,2720045,14.685417,035-9738


In [0]:
print('Average length of stay for a diabetic patient ',stay_diab['lengthofstay'].mean(),'days')

Average length of stay for a diabetic patient  4.6079179110192205 days
